<a href="https://colab.research.google.com/github/mgamzec/GeoPython/blob/main/geemap_filter_gt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=TDZ5dKZd5uceFvs-aoa3u5X2ZXe1nYCUmsqFxoRzwCs&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhSBP_M0YXoXAfqTlTtadJFWKfk9b8hoUx8to9Bqnvgcoh19aB-Nms

Successfully saved authorization token.


In [ ]:
# Installs geemap package
import google.colab
import geemap.eefolium as geemap

In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

khm=ee.FeatureCollection("WCMC/WDPA/current/polygons").filter(ee.Filter.eq('ISO3','KHM'))

#Map.centerObject(peru, 4)
Map.centerObject(khm,4)
#Map.addLayer(peru, {}, 'Countries')
Map.addLayer(khm,{},'Cambodia PAs')

Map.addLayerControl()  #This line is not needed for ipyleaflet-based Map
Map

Map(center=[12.919618870055311, 105.1162912464022], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
print("The number of PAs: ", khm.size().getInfo())

The number of PAs:  68


Total Area of Cambodia's PAs

In [ ]:
pop = khm.aggregate_sum('REP_AREA')


print("Total population: ", pop.getInfo())

Total population:  74644.81660999998


In [ ]:
# Filter to get only larger PAs
largeSheds = khm.filter(ee.Filter.gt('GIS_AREA', 2600))

# Check the number of PAs after filtering for size
print('Count after filtering by size:', largeSheds.size().getInfo())

Count after filtering by size: 11


In [ ]:
#Map.centerObject(peru, 4)
Map.centerObject(largeSheds,4)
#Map.addLayer(peru, {}, 'Countries')
Map.addLayer(largeSheds,{},'Cambodia PAs')

Map.addLayerControl()  #This line is not needed for ipyleaflet-based Map
Map

Map(bottom=4089.0, center=[13.048118389119182, 105.25211295642845], controls=(WidgetControl(options=['position…

Grouping Features

In [ ]:
sums = khm.filter(ee.Filter.And(ee.Filter.neq('GIS_AREA', None),
                                     ee.Filter.neq('STATUS_YR', None)))\
               .reduceColumns(**{
                  'selectors': ['GIS_AREA', 'STATUS_YR', 'NAME'],
                  'reducer': ee.Reducer.mean().repeat(2).group(**{
                      'groupField': 2,
                      'groupName': 'PA'})})

# Print the resultant Dictionary.
print(sums.getInfo())



{'groups': [{'PA': 'Ang Trapeng Thmor', 'mean': [126.987688809426, 2016]}, {'PA': 'Angkor', 'mean': [137.985502869213, 1993]}, {'PA': 'Anlong Pring', 'mean': [2.18663138373544, 2016]}, {'PA': 'Banteay Chhmar', 'mean': [850.955205186975, 1993]}, {'PA': 'Boeng Chhmar Core Area (Ramsar Site)', 'mean': [146.250437563593, 2000]}, {'PA': 'Boeng Lum Kud', 'mean': [1.11265142300564, 2018]}, {'PA': 'Boeng Paer', 'mean': [2508.39527946083, 1993]}, {'PA': 'Boeng Prek Lpov', 'mean': [83.4511761409677, 2016]}, {'PA': 'Boeng Prek Tub', 'mean': [0.163796004073936, 2016]}, {'PA': 'Boeng Yakaom Yakkara', 'mean': [1.32923958652482, 2020]}, {'PA': 'Boeng Yeak Laom', 'mean': [2.25957980767951, 2018]}, {'PA': 'Botum Sakor', 'mean': [1475.44559442819, 1993]}, {'PA': 'Cardamom Corridor', 'mean': [1701.63200922221, 2017]}, {'PA': 'Central Kravanh', 'mean': [4026.93874458258, 2016]}, {'PA': 'Chhaeb', 'mean': [1908.72737430326, 2016]}, {'PA': 'Dang Peaeng', 'mean': [286.952424977256, 1993]}, {'PA': 'Elettaria c

convert string to number

In [ ]:
khm2 = khm.map(lambda f: f.set('MARINE', ee.Number.parse(f.get('MARINE'))))

In [ ]:
sums2 = khm2.filter(ee.Filter.And(ee.Filter.neq('GIS_AREA', None),
                                     ee.Filter.neq('MARINE', None)))\
               .reduceColumns(**{
                  'selectors': ['GIS_AREA', 'MARINE','IUCN_CAT'],
                  'reducer': ee.Reducer.mean().repeat(2).group(**{
                      'groupField': 2,
                      'groupName': 'PA'})})

# Print the resultant Dictionary.
print(sums2.getInfo())


{'groups': [{'PA': 'II', 'mean': [1433.6828671478934, 0.08333333333333333]}, {'PA': 'III', 'mean': [123.98916919094721, 0]}, {'PA': 'IV', 'mean': [1890.213612101182, 0.05263157894736842]}, {'PA': 'Ia', 'mean': [118.48793288663234, 0.16666666666666666]}, {'PA': 'V', 'mean': [141.31866243594428, 0.06666666666666667]}, {'PA': 'VI', 'mean': [1334.5498874559933, 0]}]}
